# Hyperparameter Tuning for Sensor Circuit Models

This notebook helps data scientists run hyperparameter tuning experiments on individual circuits.

**Workflow:**
1. Connect to Azure ML workspace
2. Select a circuit to tune
3. Submit hyperparameter tuning job
4. Monitor progress
5. Retrieve best hyperparameters
6. Update `circuits.yaml` with best parameters

**Important:** This is for exploratory tuning only. Results must be manually applied to `circuits.yaml` and committed via PR.

## 1. Setup and Configuration

In [ ]:
# Import required libraries
from azure.ai.ml import MLClient, load_component
from azure.identity import DefaultAzureCredential
import sys
sys.path.append('..')
from scripts.hyperparameter_helper import (
    submit_tuning_job,
    get_best_trial_results,
    generate_circuits_yaml_snippet,
    compare_tuning_runs
)

In [ ]:
# Azure ML workspace configuration
subscription_id = "YOUR_SUBSCRIPTION_ID"
resource_group = "YOUR_RESOURCE_GROUP"
workspace_name = "dev-ml-workspace"  # Use dev workspace for experiments

# Connect to workspace
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)

print(f"✅ Connected to workspace: {workspace_name}")

## 2. Select Circuit and Data

In [ ]:
# Circuit to tune
plant_id = "PLANT001"
circuit_id = "CIRCUIT01"

# Paths
circuit_config_path = f"../config/circuits/{plant_id}_{circuit_id}.yaml"

# Training data asset (must be pre-registered)
# Format: azureml:sensor_training_data_{plant_id}_{circuit_id}:{version}
training_data_asset = f"azureml:sensor_training_data_{plant_id}_{circuit_id}:1"

print(f"Circuit: {plant_id}/{circuit_id}")
print(f"Config: {circuit_config_path}")
print(f"Data Asset: {training_data_asset}")

## 3. Configure Hyperparameter Tuning

In [ ]:
# Tuning configuration
max_trials = 20  # Start with 10-20 for initial experiments
sampling_algorithm = "random"  # Options: "random" or "bayesian"

# Note: Search space is defined in the component.yaml
# Default search space:
# - lstm_units: [32, 64, 128, 256]
# - learning_rate: loguniform(0.0001, 0.01)
# - epochs: [30, 50, 100]
# - batch_size: [16, 32, 64]

print(f"Max Trials: {max_trials}")
print(f"Sampling: {sampling_algorithm}")
print(f"\n⚠️  This will run {max_trials} training jobs. Estimated time: {max_trials * 10} - {max_trials * 30} minutes")

## 4. Submit Hyperparameter Tuning Job

In [ ]:
# Submit tuning job
job = submit_tuning_job(
    ml_client=ml_client,
    plant_id=plant_id,
    circuit_id=circuit_id,
    circuit_config_path=circuit_config_path,
    training_data_asset=training_data_asset,
    max_trials=max_trials,
    sampling_algorithm=sampling_algorithm
)

print(f"\n🚀 Job submitted: {job.name}")
print(f"\n📊 Monitor progress in Azure ML Studio:")
print(job.studio_url)

## 5. Monitor Job Progress

In [ ]:
# Check job status
job_name = job.name  # Or paste job name from previous run

current_job = ml_client.jobs.get(job_name)
print(f"Job Status: {current_job.status}")
print(f"Studio URL: {current_job.studio_url}")

# Wait for completion (optional)
# ml_client.jobs.stream(job_name)

## 6. Retrieve Best Hyperparameters

In [ ]:
# Get best trial results (run after job completes)
job_name = job.name  # Or paste completed job name

best_results = get_best_trial_results(ml_client, job_name)

if best_results:
    print("\n" + "="*60)
    print("BEST HYPERPARAMETERS")
    print("="*60)
    print(f"\nBest Run ID: {best_results['best_run_id']}")
    print(f"Val Loss: {best_results['best_metric_value']:.4f}")
    print(f"\nHyperparameters:")
    for param, value in best_results['hyperparameters'].items():
        print(f"  {param}: {value}")
    print(f"\nMetrics:")
    for metric, value in best_results['metrics'].items():
        if value is not None:
            print(f"  {metric}: {value:.4f}")
    print("="*60)

## 7. Generate Config Snippet for circuits.yaml

In [ ]:
# Generate YAML snippet to update circuits.yaml
if best_results:
    snippet = generate_circuits_yaml_snippet(
        plant_id=plant_id,
        circuit_id=circuit_id,
        best_params=best_results['hyperparameters']
    )
    
    print("\n📝 Next Steps:")
    print("1. Copy the snippet above")
    print(f"2. Update config/circuits.yaml for {plant_id}/{circuit_id}")
    print("3. Create a PR with the changes")
    print("4. PR will trigger training with new hyperparameters")

## 8. Compare Multiple Tuning Runs (Optional)

In [ ]:
# Compare results from multiple tuning runs
job_names = [
    "job_name_1",
    "job_name_2",
    "job_name_3"
]

compare_tuning_runs(ml_client, job_names)

## 9. Advanced: Load Component Directly

In [ ]:
# Alternative: Load component directly and customize
from azure.ai.ml import load_component

tuning_component = load_component(
    source="../components/hyperparameter-tuning-pipeline/component.yaml"
)

# Create custom job
custom_job = tuning_component(
    circuit_config=circuit_config_path,
    training_data=training_data_asset,
    max_trials=10,  # Fewer trials for quick test
    sampling_algorithm="bayesian"  # Try Bayesian optimization
)

custom_job.display_name = f"Custom_HPO_{plant_id}_{circuit_id}"
custom_job.experiment_name = "hyperparameter-tuning-experiments"

# Submit
# submitted_job = ml_client.jobs.create_or_update(custom_job)
# print(f"Submitted: {submitted_job.studio_url}")

---

## Tips and Best Practices

### Starting Small
- Begin with 5-10 trials to test the setup
- Use a subset of circuits (5-10 representative ones)
- Apply learnings to all circuits

### Sampling Algorithms
- **Random**: Good for initial exploration, parallelizes well
- **Bayesian**: Better convergence with fewer trials, sequential

### Search Space
- Start with 2-3 values per parameter
- Expand based on initial results
- Use loguniform for learning rates

### Cost Optimization
- Use fewer epochs (30-50) during tuning
- Limit concurrent trials to 3-5
- Set timeout to prevent runaway jobs

### Early Termination
- Bandit policy stops poor-performing trials early
- Saves compute time and cost
- Configured in component.yaml

---